In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import os
import seaborn as sns
import multiprocessing as mp
import gc
from scipy.stats import kurtosis, iqr, skew

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

# TRANSFORM 4 TABLES: CREDIT_CARD_BALANCE, INSTALLMENTS_PAYMENTS, POS_CASH and PREVIOUS_APPLICATION

In this section, we encode and create new features for these four tables.

In [2]:
def add_features_in_group(features, gr_, feature_name, aggs, prefix):
    '''
    Add new features to a dataframe groupby object.
        Input:
            features : dict
                Dictionary to store new features.
            gr_ : pandas.DataFrame
                Dataframe groupby object.
            feature_name : str
                Column name to calculate statistics.
            aggs : list
                List of method to calculate statistics.
            prefix : str
                Prefix of column names.
        Output:
            features : dict
                Dictionary with new features.
        '''
    for agg in aggs:
        if agg == 'sum':
            features[f'{prefix}{feature_name}_sum'] = gr_[
                feature_name].sum()
        elif agg == 'mean':
            features[f'{prefix}{feature_name}_mean'] = gr_[
                feature_name].mean()
        elif agg == 'max':
            features[f'{prefix}{feature_name}_max'] = gr_[
                feature_name].max()
        elif agg == 'min':
            features[f'{prefix}{feature_name}_min'] = gr_[
                feature_name].min()
        elif agg == 'std':
            features[f'{prefix}{feature_name}_std'] = gr_[
                feature_name].std()
        elif agg == 'count':
            features[f'{prefix}{feature_name}_count'] = gr_[
                feature_name].count()
        elif agg == 'skew':
            features[f'{prefix}{feature_name}_skew'] = skew(
                gr_[feature_name])
        elif agg == 'kurt':
            features[f'{prefix}{feature_name}_kurt'] = kurtosis(
                gr_[feature_name])
        elif agg == 'iqr':
            features[f'{prefix}{feature_name}_iqr'] = iqr(
                gr_[feature_name])
        elif agg == 'median':
            features[f'{prefix}{feature_name}_median'] = gr_[
                feature_name].median()
    return features


def installments_last_loan_features(gr):
    '''
    Calculate features for the last loan in installments_payments.csv.
        Input:
            gr : pandas.DataFrame
                DataFrame groupby object.
        Output:
            features : dict
                Dictionary with new features.
    '''
    gr_ = gr.copy()
    gr_.sort_values(['DAYS_INSTALMENT'], ascending=False, inplace=True)
    last_installment_id = gr_['SK_ID_PREV'].iloc[0]
    gr_ = gr_[gr_['SK_ID_PREV'] == last_installment_id]

    features = {}
    features = add_features_in_group(features, gr_, 'DPD',
                                     ['sum', 'mean', 'max', 'std'],
                                     'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'LATE_PAYMENT',
                                     ['count', 'mean'],
                                     'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'PAID_OVER_AMOUNT',
                                     ['sum', 'mean', 'max', 'min', 'std'],
                                     'LAST_LOAN_')
    features = add_features_in_group(features, gr_, 'PAID_OVER',
                                     ['count', 'mean'],
                                     'LAST_LOAN_')
    return features


def add_ratios_features(df):
    '''
    Calculate several ratios for the main dataset.
        Input:
            df : pandas.DataFrame
                Dataframe after merge with all other dataframes.
        Output:
            df : pandas.DataFrame
                Final ataframe with ratios added.
    '''

    # CREDIT TO INCOME RATIO
    df['BUREAU_INCOME_CREDIT_RATIO'] = df['BUREAU_AMT_CREDIT_SUM_MEAN'] / \
        df['AMT_INCOME_TOTAL']
    df['BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO'] = df['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM'] / \
        df['AMT_INCOME_TOTAL']

    # PREVIOUS TO CURRENT CREDIT RATIO
    df['CURRENT_TO_APPROVED_CREDIT_MIN_RATIO'] = df['APPROVED_AMT_CREDIT_MIN'] / \
        df['AMT_CREDIT']
    df['CURRENT_TO_APPROVED_CREDIT_MAX_RATIO'] = df['APPROVED_AMT_CREDIT_MAX'] / \
        df['AMT_CREDIT']
    df['CURRENT_TO_APPROVED_CREDIT_MEAN_RATIO'] = df['APPROVED_AMT_CREDIT_MEAN'] / df['AMT_CREDIT']

    # PREVIOUS TO CURRENT ANNUITY RATIO
    df['CURRENT_TO_APPROVED_ANNUITY_MAX_RATIO'] = df['APPROVED_AMT_ANNUITY_MAX'] / \
        df['AMT_ANNUITY']
    df['CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO'] = df['APPROVED_AMT_ANNUITY_MEAN'] / \
        df['AMT_ANNUITY']
    df['PAYMENT_MIN_TO_ANNUITY_RATIO'] = df['INS_AMT_PAYMENT_MIN'] / df['AMT_ANNUITY']
    df['PAYMENT_MAX_TO_ANNUITY_RATIO'] = df['INS_AMT_PAYMENT_MAX'] / df['AMT_ANNUITY']
    df['PAYMENT_MEAN_TO_ANNUITY_RATIO'] = df['INS_AMT_PAYMENT_MEAN'] / df['AMT_ANNUITY']

    # PREVIOUS TO CURRENT CREDIT TO ANNUITY RATIO
    df['CTA_CREDIT_TO_ANNUITY_MAX_RATIO'] = df['APPROVED_CREDIT_TO_ANNUITY_RATIO_MAX'] / \
        df['CREDIT_TO_ANNUITY_RATIO']
    df['CTA_CREDIT_TO_ANNUITY_MEAN_RATIO'] = df['APPROVED_CREDIT_TO_ANNUITY_RATIO_MEAN'] / \
        df['CREDIT_TO_ANNUITY_RATIO']

    # DAYS DIFFERENCES AND RATIOS
    df['DAYS_DECISION_MEAN_TO_BIRTH'] = df['APPROVED_DAYS_DECISION_MEAN'] / \
        df['DAYS_BIRTH']
    df['DAYS_CREDIT_MEAN_TO_BIRTH'] = df['BUREAU_DAYS_CREDIT_MEAN'] / df['DAYS_BIRTH']
    df['DAYS_DECISION_MEAN_TO_EMPLOYED'] = df['APPROVED_DAYS_DECISION_MEAN'] / \
        df['DAYS_EMPLOYED']
    df['DAYS_CREDIT_MEAN_TO_EMPLOYED'] = df['BUREAU_DAYS_CREDIT_MEAN'] / \
        df['DAYS_EMPLOYED']
    return df


In [3]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """Create a new column for each categorical value in categorical columns using get dummies. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [
            col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns,
                        dummy_na=nan_as_category)
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns


In [4]:
def group(df_to_agg, prefix, aggregations, aggregate_by='SK_ID_CURR'):
    agg_df = df_to_agg.groupby(aggregate_by).agg(aggregations)
    agg_df.columns = pd.Index(['{}{}_{}'.format(prefix, e[0], e[1].upper())
                               for e in agg_df.columns.tolist()])
    return agg_df.reset_index()


def group_and_merge(df_to_agg, df_to_merge, prefix, aggregations, aggregate_by='SK_ID_CURR'):
    agg_df = group(df_to_agg, prefix, aggregations, aggregate_by=aggregate_by)
    return df_to_merge.merge(agg_df, how='left', on=aggregate_by)
PREVIOUS_AGG = {
    'SK_ID_PREV': ['nunique'],

    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean', 'var'],

    'CNT_PAYMENT': ['max', 'mean'],
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],

    'DAYS_TERMINATION': ['max'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'DOWN_PAYMENT_TO_CREDIT': ['mean'],

    'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],

    'NEW_CREDIT_GOODS_RATE': ['mean', 'max', 'var', 'min'],
    'NEW_END_DIFF': ['min', 'max'],
    'NEW_DAYS_DUE_DIFF': ['min', 'max'],
    'NEW_RETURN_DAY': ['min', 'max'],

    'RATE_DOWN_PAYMENT': ['max', 'mean'],
}

PREVIOUS_ACTIVE_AGG = {
    'SK_ID_PREV': ['nunique'],

    'AMT_ANNUITY': ['max', 'sum'],
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['sum'],
    'AMT_DOWN_PAYMENT': ['max', 'mean'],
    'AMT_PAYMENT': ['sum'],

    'CNT_PAYMENT': ['mean', 'sum'],

    'DAYS_DECISION': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],

    'INSTALMENT_PAYMENT_DIFF': ['mean', 'max'],

    'REMAINING_DEBT': ['max', 'mean', 'sum'],
    'REPAYMENT_RATIO': ['mean'],

    'SIMPLE_INTERESTS': ['mean'],
}

PREVIOUS_APPROVED_AGG = {
    'SK_ID_PREV': ['nunique'],

    'AMT_ANNUITY': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean'],
    'AMT_DOWN_PAYMENT': ['max'],
    'AMT_GOODS_PRICE': ['max'],
    'AMT_INTEREST': ['min', 'max', 'mean'],
    'APPLICATION_CREDIT_DIFF': ['max'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],

    'CNT_PAYMENT': ['max', 'mean'],
    'CREDIT_TO_ANNUITY_RATIO': ['mean', 'max'],

    'DAYS_DECISION': ['min', 'mean'],
    'DAYS_TERMINATION': ['mean'],
    'DAYS_FIRST_DRAWING': ['max', 'mean'],
    'DAYS_FIRST_DUE': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE': ['max', 'mean'],
    'DAYS_LAST_DUE_DIFF': ['min', 'max', 'mean'],

    'HOUR_APPR_PROCESS_START': ['min', 'max'],

    'INTEREST_SHARE': ['min', 'max', 'mean'],
    'INTEREST_RATE': ['min', 'max', 'mean'],

    'SIMPLE_INTERESTS': ['min', 'max', 'mean'],
}

PREVIOUS_REFUSED_AGG = {
    'AMT_APPLICATION': ['max', 'mean'],
    'AMT_CREDIT': ['min', 'max'],
    'APPLICATION_CREDIT_DIFF': ['min', 'max', 'mean', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'mean'],

    'CNT_PAYMENT': ['max', 'mean'],

    'DAYS_DECISION': ['min', 'max', 'mean'],

    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

PREVIOUS_LATE_PAYMENTS_AGG = {
    'APPLICATION_CREDIT_DIFF': ['min'],

    'DAYS_DECISION': ['min', 'max', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],

    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],
}

PREVIOUS_LOAN_TYPE_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['min', 'var'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],

    'DAYS_DECISION': ['max'],
    'DAYS_LAST_DUE_1ST_VERSION': ['max', 'mean'],

    'SIMPLE_INTERESTS': ['min', 'mean', 'max', 'var'],
}
PREVIOUS_TIME_AGG = {
    'AMT_CREDIT': ['sum'],
    'AMT_ANNUITY': ['mean', 'max'],
    'APPLICATION_CREDIT_DIFF': ['min'],
    'APPLICATION_CREDIT_RATIO': ['min', 'max', 'mean'],

    'DAYS_DECISION': ['min', 'mean'],
    'DAYS_LAST_DUE_1ST_VERSION': ['min', 'max', 'mean'],

    'NAME_CONTRACT_TYPE_Consumer loans': ['mean'],
    'NAME_CONTRACT_TYPE_Cash loans': ['mean'],
    'NAME_CONTRACT_TYPE_Revolving loans': ['mean'],

    'SIMPLE_INTERESTS': ['mean', 'max'],
}


In [5]:
prev_application = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\previous_application.csv")
installments_payments = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\installments_payments.csv")
credit_card_balance = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\credit_card_balance.csv")
pos_cash_balance = pd.read_csv(r"C:\Users\ADMIN\Downloads\Project1-HOMECREDIT\home-credit-default-risk\POS_CASH_balance.csv")

## PREVIOUS_APPLICATION

In [ ]:
def previous_application(pre_app, install_pay):
    """ Process mainly on previous_application.csv and and merge with 
    some columns of installments_payments.csv for insights return a pandas dataframe. """
    prev = pre_app
    pay = install_pay

    # One-hot encode most important categorical features
    enc_columns = [
        'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE', 'CHANNEL_TYPE',
        'NAME_TYPE_SUITE', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
        'NAME_PRODUCT_TYPE', 'NAME_CLIENT_TYPE']
    prev, categorical_cols = one_hot_encoder(
        prev, enc_columns, nan_as_category=False)

    new_coding = {"0": "Yes", "1": "No"}
    # Calculate ratios and difference for some columns
    prev['APPLICATION_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT'] #chênh lệch tiền muốn vat và tiền được duyệt
    prev['APPLICATION_CREDIT_RATIO'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT'] # tỷ lệ

    prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT']/prev['AMT_ANNUITY'] #Số tiền vay/số tiền trả mỗi kì

    prev['DOWN_PAYMENT_TO_CREDIT'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT'] #% đặt cọc so với khoản vay
    prev["NEW_APP_CREDIT_RATE_RATIO"] = prev["APPLICATION_CREDIT_RATIO"].apply(
        lambda x: 1 if (x <= 1) else 0) #số tiền vay có bé hơn số tiền được duyệt hay không
    prev['NEW_APP_CREDIT_RATE_RATIO'] = prev['NEW_APP_CREDIT_RATE_RATIO'].astype(
        'O')
    prev['NEW_APP_CREDIT_RATE_RATIO'] = prev['NEW_APP_CREDIT_RATE_RATIO'].replace(
        new_coding)

    prev["NEW_CNT_PAYMENT"] = pd.cut(x=prev['CNT_PAYMENT'], bins=[
                                     0, 12, 60, 120], labels=["Short", "Middle", "Long"]) #phân loại vay ngắn/trung/dài hạn
    prev['NEW_CREDIT_GOODS_RATE'] = prev['AMT_CREDIT'] / prev['AMT_GOODS_PRICE'] #vay bao nhiêu % giá trị món hàng

    prev["NEW_END_DIFF"] = prev["DAYS_TERMINATION"] - prev["DAYS_LAST_DUE"] #ngày kết thúc hợp đồng và ngày trả cuối
    prev['NEW_DAYS_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - \
        prev['DAYS_FIRST_DUE'] #chênh lệch kì trả đầu tiên và kì trả cuối
    prev['NEW_RETURN_DAY'] = prev['DAYS_DECISION'] + prev['CNT_PAYMENT'] * 30 #ngày dự kiến trả
    prev['NEW_DAYS_TERMINATION_DIFF'] = prev['DAYS_TERMINATION'] - \
        prev['NEW_RETURN_DAY'] #chênh lệch giữa ngày kết thúc thực tế và ngày dự kiến

    prev['NFLAG_LAST_APPL_IN_DAY'] = prev['NFLAG_LAST_APPL_IN_DAY'].astype("O")

    # Interest ratio on previous application
    total_payment = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    prev['SIMPLE_INTERESTS'] = (
        total_payment/prev['AMT_CREDIT'] - 1)/prev['CNT_PAYMENT'] #Interest rate
    prev['AMT_INTEREST'] = prev['CNT_PAYMENT'] * \
        prev['AMT_ANNUITY'] - prev['AMT_CREDIT'] #total amount to repay
    prev['INTEREST_SHARE'] = prev['AMT_INTEREST'] / \
        (prev['AMT_CREDIT'] + 0.00001)  # smoothing to avoid division by zero -> %gross fund
    prev['INTEREST_RATE'] = 2 * 12 * prev['AMT_INTEREST'] / \
        (prev['AMT_CREDIT'] * (prev['CNT_PAYMENT'] + 1))

    # Active loans - approved and not complete yet (last_due 365243)
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    active_df = approved[approved['DAYS_LAST_DUE'] == 365243]

    # Find how much was already payed in active loans (using installments csv)
    active_pay = pay[pay['SK_ID_PREV'].isin(active_df['SK_ID_PREV'])]
    active_pay_agg = active_pay.groupby(
        'SK_ID_PREV')[['AMT_INSTALMENT', 'AMT_PAYMENT']].sum()
    active_pay_agg.reset_index(inplace=True)

    # Active loans: difference of what was payed and installments
    active_pay_agg['INSTALMENT_PAYMENT_DIFF'] = active_pay_agg['AMT_INSTALMENT'] - \
        active_pay_agg['AMT_PAYMENT']

    # Merge with active_df
    active_df = active_df.merge(active_pay_agg, on='SK_ID_PREV', how='left')
    active_df['REMAINING_DEBT'] = active_df['AMT_CREDIT'] - \
        active_df['AMT_PAYMENT']
    active_df['REPAYMENT_RATIO'] = active_df['AMT_PAYMENT'] / \
        active_df['AMT_CREDIT']

    # Perform aggregations for active applications
    active_agg_df = group(active_df, 'PREV_ACTIVE_', PREVIOUS_ACTIVE_AGG)
    active_agg_df['TOTAL_REPAYMENT_RATIO'] = active_agg_df['PREV_ACTIVE_AMT_PAYMENT_SUM'] /\
        active_agg_df['PREV_ACTIVE_AMT_CREDIT_SUM']
    del active_pay, active_pay_agg, active_df

    # Change 365.243 values to nan (missing)
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

    # Days last due difference (scheduled x done)
    prev['DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - \
        prev['DAYS_LAST_DUE']
    approved['DAYS_LAST_DUE_DIFF'] = approved['DAYS_LAST_DUE_1ST_VERSION'] - \
        approved['DAYS_LAST_DUE']

    # Categorical features
    categorical_agg = {key: ['mean'] for key in categorical_cols}

    # Perform general aggregations
    agg_prev = group(prev, 'PREV_', {**PREVIOUS_AGG, **categorical_agg})

    # Merge active loans dataframe on agg_prev
    agg_prev = agg_prev.merge(active_agg_df, how='left', on='SK_ID_CURR')
    del active_agg_df

    # Aggregations for approved and refused loans
    agg_prev = group_and_merge(
        approved, agg_prev, 'APPROVED_', PREVIOUS_APPROVED_AGG)
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    agg_prev = group_and_merge(
        refused, agg_prev, 'REFUSED_', PREVIOUS_REFUSED_AGG)
    del approved, refused

    # Aggregations for Consumer loans and Cash loans -> Feature based on type of loan
    for loan_type in ['Consumer loans', 'Cash loans']:
        type_df = prev[prev[f'NAME_CONTRACT_TYPE_{loan_type}'] == 1]
        prefix = 'PREV_' + loan_type.split(" ", maxsplit=1)[0] + '_'
        agg_prev = group_and_merge(
            type_df, agg_prev, prefix, PREVIOUS_LOAN_TYPE_AGG)
    del type_df

    # Get the SK_ID_PREV for loans with late payments (days past due) -> Does customer pay late
    pay['LATE_PAYMENT'] = pay['DAYS_ENTRY_PAYMENT'] - pay['DAYS_INSTALMENT']
    pay['LATE_PAYMENT'] = pay['LATE_PAYMENT'].apply(
        lambda x: 1 if x > 0 else 0)
    dpd_id = pay[pay['LATE_PAYMENT'] > 0]['SK_ID_PREV'].unique()

    # Aggregations for loans with late payments 
    agg_dpd = group_and_merge(prev[prev['SK_ID_PREV'].isin(dpd_id)], agg_prev,
                              'PREV_LATE_', PREVIOUS_LATE_PAYMENTS_AGG)
    del agg_dpd, dpd_id

    # Aggregations for loans in the last x months -> compare loan 12m and 24m
    for time_frame in [12, 24]:
        time_frame_df = prev[prev['DAYS_DECISION'] >= -30*time_frame]
        prefix = f'PREV_LAST{time_frame}M_'
        agg_prev = group_and_merge(
            time_frame_df, agg_prev, prefix, PREVIOUS_TIME_AGG)
        del time_frame_df

    del prev
    gc.collect()
    return agg_prev


In [7]:
prev = previous_application(prev_application, installments_payments)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_37916\2227615408.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  prev['NEW_APP_CREDIT_RATE_RATIO'] = prev['NEW_APP_CREDIT_RATE_RATIO'].replace(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_37916\2227615408.py:83: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original objec

In [8]:
def do_sum(dataframe, group_cols, counted, agg_name):
    gp = dataframe[group_cols + [counted]].groupby(
        group_cols)[counted].sum().reset_index().rename(columns={counted: agg_name})
    dataframe = dataframe.merge(gp, on=group_cols, how='left')
    return dataframe

## POS_CASH

In [9]:
POS_CASH_AGG = {
    'SK_ID_PREV': ['nunique'],
    'SK_ID_CURR': ['count'],

    'EXP_CNT_INSTALMENT': ['last', 'min', 'max', 'mean', 'sum'],
    'EXP_CNT_INSTALMENT_FUTURE': ['last', 'min', 'max', 'mean', 'sum'],

    'LATE_PAYMENT': ['mean'],

    'MONTHS_BALANCE': ['min', 'max', 'size'],

    'POS_IS_DPD': ['mean', 'sum'],
    'POS_IS_DPD_UNDER_120': ['mean', 'sum'],
    'POS_IS_DPD_OVER_120': ['mean', 'sum'],

    'SK_DPD': ['max', 'mean', 'sum', 'var', 'min'],
    'SK_DPD_DEF': ['max', 'mean', 'sum'],
}

In [10]:
def pos_cash(pos_cash_balance):
    """ Process POS_CASH_balance.csv and return a pandas dataframe. """
    pos = pos_cash_balance

    # computing Exponential Moving Average for some features based on MONTHS_BALANCE
    columns_for_ema = ['CNT_INSTALMENT', 'CNT_INSTALMENT_FUTURE']
    exp_columns = ['EXP_'+ele for ele in columns_for_ema]
    pos[exp_columns] = pos.groupby('SK_ID_PREV')[columns_for_ema].transform(
        lambda x: x.ewm(alpha=0.6).mean())

    # One-hot encode categorical features
    pos, categorical_cols = one_hot_encoder(pos, nan_as_category=False)

    # Flag months with late payment
    pos['LATE_PAYMENT'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

    # Flag days past due
    pos['POS_IS_DPD'] = pos['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

    # Flag days past due less than 120 days
    pos['POS_IS_DPD_UNDER_120'] = pos['SK_DPD'].apply(
        lambda x: 1 if (x > 0) & (x < 120) else 0)

    # Flag days past due over 120 days
    pos['POS_IS_DPD_OVER_120'] = pos['SK_DPD'].apply(
        lambda x: 1 if x >= 120 else 0)

    # Aggregate by SK_ID_CURR
    categorical_agg = {key: ['mean'] for key in categorical_cols}
    pos_agg = group(pos, 'POS_', {**POS_CASH_AGG, **categorical_agg})

    # Sort and group by SK_ID_PREV
    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.groupby('SK_ID_PREV')

    # Create new dataframe to store features calculated from gp
    df = pd.DataFrame()

    df['SK_ID_CURR'] = gp['SK_ID_CURR'].first()
    df['MONTHS_BALANCE_MAX'] = gp['MONTHS_BALANCE'].max()

    # Percentage of previous loans completed and completed before initial term
    df['POS_LOAN_COMPLETED_MEAN'] = gp['NAME_CONTRACT_STATUS_Completed'].mean()
    df['POS_COMPLETED_BEFORE_MEAN'] = gp['CNT_INSTALMENT'].first() - \
        gp['CNT_INSTALMENT'].last()
    df['POS_COMPLETED_BEFORE_MEAN'] = df.apply(lambda x: 1 if x['POS_COMPLETED_BEFORE_MEAN'] > 0
                                               and x['POS_LOAN_COMPLETED_MEAN'] > 0 else 0, axis=1)

    # Number of remaining installments (future installments) and percentage from total
    df['POS_REMAINING_INSTALMENTS'] = gp['CNT_INSTALMENT_FUTURE'].last()
    df['POS_REMAINING_INSTALMENTS_RATIO'] = gp['CNT_INSTALMENT_FUTURE'].last() / \
        gp['CNT_INSTALMENT'].last()

    # Group by SK_ID_CURR and merge
    df_gp = df.groupby('SK_ID_CURR').sum().reset_index()
    df_gp.drop(['MONTHS_BALANCE_MAX'], axis=1, inplace=True)
    pos_agg = pd.merge(pos_agg, df_gp, on='SK_ID_CURR', how='left')
    del df, gp, df_gp, sort_pos

    # Percentage of late payments for the 3 most recent applications
    pos = do_sum(pos, ['SK_ID_PREV'], 'LATE_PAYMENT', 'LATE_PAYMENT_SUM')

    # Last month of each application
    last_month_df = pos.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()

    # Most recent applications (last 3)
    sort_pos = pos.sort_values(by=['SK_ID_PREV', 'MONTHS_BALANCE'])
    gp = sort_pos.iloc[last_month_df].groupby('SK_ID_CURR').tail(3)

    # Average application features over the last 3 applications
    gp_mean = gp.groupby('SK_ID_CURR').mean().reset_index()
    pos_agg = pd.merge(
        pos_agg, gp_mean[['SK_ID_CURR', 'LATE_PAYMENT_SUM']], on='SK_ID_CURR', how='left')

    # Drop some useless categorical features, which were created to calculate to other features
    drop_features = [
        'POS_NAME_CONTRACT_STATUS_Canceled_MEAN', 'POS_NAME_CONTRACT_STATUS_Amortized debt_MEAN',
        'POS_NAME_CONTRACT_STATUS_XNA_MEAN']
    pos_agg.drop(drop_features, axis=1, inplace=True)
    del gp, gp_mean, sort_pos, pos
    gc.collect()
    return pos_agg


In [11]:
pos_agg = pos_cash(pos_cash_balance)

## INSTALLMENTS_PAYMENTS

In [12]:
def installment_last_loan_features(pay):
    cols = [
        'SK_ID_CURR', 'SK_ID_PREV', 'DPD', 'LATE_PAYMENT',
        'PAID_OVER_AMOUNT', 'PAID_OVER', 'DAYS_INSTALMENT'
    ]

    gp = pay[cols].groupby('SK_ID_CURR')

    def process_group(gr):
        gr_ = gr.sort_values('DAYS_INSTALMENT', ascending=False)
        last_prev = gr_['SK_ID_PREV'].iloc[0]
        gr_ = gr_[gr_['SK_ID_PREV'] == last_prev]

        features = {}
        features = add_features_in_group(features, gr_, 'DPD',
                                         ['sum', 'mean', 'max', 'std'], 'LAST_LOAN_')
        features = add_features_in_group(features, gr_, 'LATE_PAYMENT',
                                         ['count', 'mean'], 'LAST_LOAN_')
        features = add_features_in_group(features, gr_, 'PAID_OVER_AMOUNT',
                                         ['sum', 'mean', 'max', 'min', 'std'], 'LAST_LOAN_')
        features = add_features_in_group(features, gr_, 'PAID_OVER',
                                         ['count', 'mean'], 'LAST_LOAN_')
        return pd.Series(features)

    # chạy tuần tự
    g = gp.apply(process_group).reset_index()
    return g

In [13]:
INSTALLMENTS_AGG = {
    'SK_ID_PREV': ['size', 'nunique'],

    'AMT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],

    'DAYS_ENTRY_PAYMENT': ['min', 'max', 'mean'],
    'DPD': ['max', 'mean', 'var'],
    'DBD': ['max', 'mean', 'var'],
    'DPD_7': ['mean'],
    'DPD_15': ['mean'],

    'INS_IS_DPD_UNDER_120': ['mean', 'sum'],
    'INS_IS_DPD_OVER_120': ['mean', 'sum'],

    'LATE_PAYMENT': ['mean', 'sum'],
    'LATE_PAYMENT_RATIO': ['mean'],

    'PAID_OVER': ['mean'],
    'PAYMENT_DIFFERENCE': ['mean'],
    'PAYMENT_RATIO': ['mean'],

    'SIGNIFICANT_LATE_PAYMENT': ['mean', 'sum'],
}

INSTALLMENTS_TIME_AGG = {
    'SK_ID_PREV': ['size'],

    'AMT_INSTALMENT': ['min', 'max', 'mean', 'sum'],
    'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],

    'DAYS_ENTRY_PAYMENT': ['min', 'max', 'mean'],
    'DPD': ['max', 'mean', 'var'],
    'DBD': ['max', 'mean', 'var'],
    'DPD_7': ['mean'],
    'DPD_15': ['mean'],

    'LATE_PAYMENT': ['mean'],
    'LATE_PAYMENT_RATIO': ['mean'],

    'PAYMENT_DIFFERENCE': ['mean'],
    'PAYMENT_RATIO': ['mean'],

    'SIGNIFICANT_LATE_PAYMENT': ['mean'],
}


In [ ]:
def installment(ins):
    pay = ins.copy()

    # Payment group sum
    pay = do_sum(pay, ['SK_ID_PREV', 'NUM_INSTALMENT_NUMBER'],
                 'AMT_PAYMENT', 'AMT_PAYMENT_GROUPED')

    # Core features
    pay['PAYMENT_DIFFERENCE'] = pay['AMT_INSTALMENT'] - pay['AMT_PAYMENT_GROUPED']
    pay['PAYMENT_RATIO'] = pay['AMT_INSTALMENT'] / pay['AMT_PAYMENT_GROUPED']
    pay['PAID_OVER_AMOUNT'] = pay['AMT_PAYMENT'] - pay['AMT_INSTALMENT']
    pay['PAID_OVER'] = (pay['PAID_OVER_AMOUNT'] > 0).astype(int)

    # DPD / DBD
    pay['DPD'] = (pay['DAYS_ENTRY_PAYMENT'] - pay['DAYS_INSTALMENT']).clip(lower=0)
    pay['DBD'] = (pay['DAYS_INSTALMENT'] - pay['DAYS_ENTRY_PAYMENT']).clip(lower=0)

    pay['LATE_PAYMENT'] = (pay['DBD'] > 0).astype(int)
    pay['INSTALMENT_PAYMENT_RATIO'] = pay['AMT_PAYMENT'] / pay['AMT_INSTALMENT']
    pay['LATE_PAYMENT_RATIO'] = pay['INSTALMENT_PAYMENT_RATIO'] * pay['LATE_PAYMENT']
    pay['SIGNIFICANT_LATE_PAYMENT'] = (pay['LATE_PAYMENT_RATIO'] > 0.05).astype(int)

    pay['DPD_7'] = (pay['DPD'] >= 7).astype(int)
    pay['DPD_15'] = (pay['DPD'] >= 15).astype(int)
    pay['INS_IS_DPD_UNDER_120'] = ((pay['DPD'] > 0) & (pay['DPD'] < 120)).astype(int)
    pay['INS_IS_DPD_OVER_120'] = (pay['DPD'] >= 120).astype(int)

    # Aggregations
    pay_agg = group(pay, 'INS_', INSTALLMENTS_AGG)

    # Last 24/60 months
    for months in [24, 60]:
        mask = pay['DAYS_INSTALMENT'] >= -30 * months
        prev = pay.loc[mask, 'SK_ID_PREV'].unique()
        pay_recent = pay[pay['SK_ID_PREV'].isin(prev)]
        prefix = f'INS_{months}M_'
        pay_agg = group_and_merge(pay_recent, pay_agg,
                                  prefix, INSTALLMENTS_TIME_AGG)

    # Last loan features
    g = installment_last_loan_features(pay)
    pay_agg = pay_agg.merge(g, on='SK_ID_CURR', how='left')

    gc.collect()
    return pay_agg

In [15]:
install_pay = installment(installments_payments)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_37916\3585989494.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  g = gp.apply(process_group).reset_index()


## CREDIT_CARD_BALANCE

In [ ]:
CREDIT_CARD_AGG = {
    'AMT_BALANCE': ['sum', 'min', 'max'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['max', 'sum', 'min'],
    'AMT_DRAWINGS_ATM_CURRENT': ['max', 'sum'],
    'AMT_DRAWINGS_CURRENT': ['max', 'sum'],
    'AMT_DRAWINGS_POS_CURRENT': ['max', 'sum'],
    'AMT_DRAWING_SUM': ['sum', 'max'],
    'AMT_INST_MIN_REGULARITY': ['max', 'mean', 'min'],
    'AMT_INTEREST_RECEIVABLE': ['min', 'mean'],
    'AMT_PAYMENT_TOTAL_CURRENT': ['max', 'mean', 'sum', 'var'],
    'AMT_TOTAL_RECEIVABLE': ['max', 'mean', 'sum'],

    'BALANCE_LIMIT_RATIO': ['mean', 'max', 'min'],

    'CNT_DRAWING_SUM': ['sum', 'max'],
    'CNT_DRAWINGS_ATM_CURRENT': ['max', 'mean', 'sum'],
    'CNT_DRAWINGS_CURRENT': ['max', 'mean', 'sum'],
    'CNT_DRAWINGS_POS_CURRENT': ['mean', 'sum', 'max'],

    'EXP_AMT_BALANCE': ['last'],
    'EXP_AMT_CREDIT_LIMIT_ACTUAL': ['last'],
    'EXP_AMT_RECEIVABLE_PRINCIPAL': ['last'],
    'EXP_AMT_RECEIVABLE': ['last'],
    'EXP_AMT_TOTAL_RECEIVABLE': ['last'],
    'EXP_AMT_DRAWING_SUM': ['last'],
    'EXP_BALANCE_LIMIT_RATIO': ['last'],
    'EXP_CNT_DRAWING_SUM': ['last'],
    'EXP_MIN_PAYMENT_RATIO': ['last'],
    'EXP_PAYMENT_MIN_DIFF': ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO': ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE': ['last'],
    'EXP_SK_DPD_RATIO': ['last'],

    'LIMIT_USE': ['max', 'mean'],
    'LATE_PAYMENT': ['max', 'sum'],

    'MIN_PAYMENT_RATIO': ['min', 'mean'],
    'MIN_PAYMENT_TOTAL_RATIO': ['min', 'mean'],
    'MONTHS_BALANCE': ['min'],

    'SK_DPD': ['mean', 'max', 'sum'],
    'SK_DPD_DEF': ['max', 'sum'],
    'SK_DPD_RATIO': ['max', 'mean'],

    'PAYMENT_DIV_MIN': ['min', 'mean'],
    'PAYMENT_MIN_DIFF': ['min', 'mean'],
    'PAYMENT_MIN_TOTAL_DIFF': ['min', 'mean'],
}

CREDIT_CARD_TIME_AGG = {
    'AMT_BALANCE': ['mean', 'max'],

    'CNT_DRAWINGS_ATM_CURRENT': ['mean'],

    'EXP_AMT_DRAWING_SUM': ['last'],
    'EXP_BALANCE_LIMIT_RATIO': ['last'],
    'EXP_CNT_DRAWING_SUM': ['last'],
    'EXP_MIN_PAYMENT_RATIO': ['last'],
    'EXP_PAYMENT_MIN_DIFF': ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO': ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE': ['last'],
    'EXP_SK_DPD_RATIO': ['last'],

    'LIMIT_USE': ['max', 'mean'],

    'SK_DPD': ['max', 'sum'],
}

rolling_columns = [
    'AMT_BALANCE',
    'AMT_CREDIT_LIMIT_ACTUAL',
    'AMT_DRAWING_SUM',
    'AMT_INTEREST_RECEIVABLE',
    'AMT_RECEIVABLE',
    'AMT_RECEIVABLE_PRINCIPAL',
    'AMT_TOTAL_RECEIVABLE',

    'BALANCE_LIMIT_RATIO',

    'CNT_DRAWING_SUM',

    'MIN_PAYMENT_RATIO',
    'MIN_PAYMENT_TOTAL_RATIO',

    'PAYMENT_MIN_DIFF',

    'SK_DPD_RATIO']


In [20]:

def credit_card(data):
    """ Process credit_card_balance.csv and return a pandas dataframe. """
    cc = data

    # One-hot encoder
    cc, _ = one_hot_encoder(cc, nan_as_category=False)

    # Rename columns to correct format
    cc.rename(columns={'AMT_RECIVABLE': 'AMT_RECEIVABLE'}, inplace=True)

    # Amount used from limit
    cc['LIMIT_USE'] = cc['AMT_BALANCE'] / cc['AMT_CREDIT_LIMIT_ACTUAL']

    # Current payment / Min payment
    cc['PAYMENT_DIV_MIN'] = cc['AMT_PAYMENT_CURRENT'] / \
        cc['AMT_INST_MIN_REGULARITY']

    # Late payment
    cc['LATE_PAYMENT'] = cc['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)

    # How much drawing of limit
    cc['DRAWING_LIMIT_RATIO'] = cc['AMT_DRAWINGS_ATM_CURRENT'] / \
        cc['AMT_CREDIT_LIMIT_ACTUAL']

    # Total amount of drawing at ATM
    cc['AMT_DRAWING_SUM'] = cc['AMT_DRAWINGS_ATM_CURRENT'] + cc['AMT_DRAWINGS_CURRENT'] + cc[
        'AMT_DRAWINGS_OTHER_CURRENT'] + cc['AMT_DRAWINGS_POS_CURRENT']

    # Total of drawing on previous credit
    cc['CNT_DRAWING_SUM'] = cc['CNT_DRAWINGS_ATM_CURRENT'] + cc['CNT_DRAWINGS_CURRENT'] + \
        cc['CNT_DRAWINGS_OTHER_CURRENT'] + \
        cc['CNT_DRAWINGS_POS_CURRENT'] + cc['CNT_INSTALMENT_MATURE_CUM']

    # ATM balance ratio, with smoothing by 0.00001 to avoid dividing by zero
    cc['BALANCE_LIMIT_RATIO'] = cc['AMT_BALANCE'] / \
        (cc['AMT_CREDIT_LIMIT_ACTUAL'] + 0.00001)

    # Number of times drawing was done, with smoothing by 0.00001 to avoid dividing by zero
    cc['MIN_PAYMENT_RATIO'] = cc['AMT_PAYMENT_CURRENT'] / \
        (cc['AMT_INST_MIN_REGULARITY'] + 0.0001)
    cc['MIN_PAYMENT_TOTAL_RATIO'] = cc['AMT_PAYMENT_TOTAL_CURRENT'] / \
        (cc['AMT_INST_MIN_REGULARITY'] + 0.00001)

    # Days past due ratio, with smoothing by 0.00001 to avoid dividing by zero
    cc['SK_DPD_RATIO'] = cc['SK_DPD'] / (cc['SK_DPD_DEF'] + 0.00001)

    # Difference in payment min and current
    cc['PAYMENT_MIN_DIFF'] = cc['AMT_PAYMENT_CURRENT'] - \
        cc['AMT_INST_MIN_REGULARITY']

    # Difference in total payment and current
    cc['PAYMENT_MIN_TOTAL_DIFF'] = cc['AMT_PAYMENT_TOTAL_CURRENT'] - \
        cc['AMT_INST_MIN_REGULARITY']

    # Interest received
    cc['AMT_INTEREST_RECEIVABLE'] = cc['AMT_TOTAL_RECEIVABLE'] - \
        cc['AMT_RECEIVABLE_PRINCIPAL']

    # calculating the rolling Exponential Weighted Moving Average over months for certain features
    exp_weighted_columns = ['EXP_' + ele for ele in rolling_columns]
    cc[exp_weighted_columns] = cc.groupby(['SK_ID_CURR', 'SK_ID_PREV'])[
        rolling_columns].transform(lambda x: x.ewm(alpha=0.7).mean())

    # Aggregations by SK_ID_CURR
    cc_agg = cc.groupby('SK_ID_CURR').agg(CREDIT_CARD_AGG)
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper()
                              for e in cc_agg.columns.tolist()])
    cc_agg.reset_index(inplace=True)

    # Last month balance of each credit card application
    last_ids = cc.groupby('SK_ID_PREV')['MONTHS_BALANCE'].idxmax()
    last_months_df = cc[cc.index.isin(last_ids)]
    cc_agg = group_and_merge(last_months_df, cc_agg, 'CC_LAST_', {
                             'AMT_BALANCE': ['mean', 'max']})

    # Aggregations for last x months
    for months in [12, 24, 48]:
        cc_prev_id = cc[cc['MONTHS_BALANCE'] >= -months]['SK_ID_PREV'].unique()
        cc_recent = cc[cc['SK_ID_PREV'].isin(cc_prev_id)]
        prefix = f'INS_{months}M_'
        cc_agg = group_and_merge(
            cc_recent, cc_agg, prefix, CREDIT_CARD_TIME_AGG)

    del cc_recent, cc_prev_id, last_months_df, last_ids, cc
    gc.collect()
    return cc_agg


In [21]:
cc = credit_card(credit_card_balance)

## Merge

In [22]:
merge_data = pd.merge(prev, cc, on='SK_ID_CURR', how='left')
print(merge_data.shape)

(338857, 364)


In [23]:
merge_data = pd.merge(merge_data, pos_agg, on='SK_ID_CURR', how='left')
print(merge_data.shape)

(338857, 405)


In [24]:
merge_data = pd.merge(merge_data, install_pay, on='SK_ID_CURR', how='left')
print(merge_data.shape)

(338857, 501)


In [27]:
merge_data.to_csv("feature_prev_cc_pos_install.csv", index = False)